**runs gpu-accelerated leiden clustering with cugraph**
- separate notebook due to compat issues w/ scvi

In [ ]:
!pip install --force-reinstall cudf-cu12 cugraph-cu12 cuml-cu12 --extra-index-url=https://pypi.nvidia.com


In [ ]:
import cupy as cp
import cudf, cuml, cugraph, pickle
import cugraph
import pickle

with open('/content/drive/MyDrive/project_output/u_embeddings.pkl', 'rb') as f:
    X = pickle.load(f)

X = cp.asarray(Z)

# knn
knn = cuml.NearestNeighbors(n_neighbors=15)
knn.fit(X)
distances, indices = knn.kneighbors(X)

# edges cudf
src = cp.repeat(cp.arange(X.shape[0]), indices.shape[1])
dst = indices.reshape(-1)
gdf = cudf.DataFrame({"src": src, "dst": dst})

# cugraph
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source="src", destination="dst", renumber=False)

# leiden
parts = cugraph.leiden(G, resolution=0.2)

print(parts[0].head())


In [ ]:
parts[0].to_csv('/content/parts.res0_2.tsv', sep = '\t')